In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

In [4]:
def get_new_col(df, new_col):
  cal_train = df.copy(deep=True)
  for i in new_col:
      split_col = i.split('*')
      if len(split_col) >= 2:
          col_name = ""
          for j in range(len(split_col)):
              if j == 0:
                  col_name = split_col[j]
                  cal_train['cal'] = cal_train[split_col[j]]
              else:
                  col_name += f'*{split_col[j]}'
                  cal_train['cal'] *= cal_train[split_col[j]]
          df[col_name] = cal_train['cal']
  return df

In [5]:
def get_data(data_list):
    copy_data_list = copy.deepcopy(data_list)
    new_data = ""
    for i in range(len(copy_data_list)):
        if i == 0:
            new_data = copy_data_list[i]
        else:
            new_data = pd.concat([new_data, copy_data_list[i]])
    del_list = []
    for i in new_data.columns:
        if '남은시간' in i:
            del_list.append(i)
    new_data = new_data.drop(del_list, axis=1)
    return new_data

In [6]:
def test_(df):
    test_train = df.copy(deep=True)
    test_train['풍속'] = np.log1p(test_train['풍속'])

    y_train = test_train['풍속']
    X = test_train.drop(['풍속'],axis=1)
    X_col = X.columns
    scaler = StandardScaler()
    scaler.fit(X)
    scaled_X_train  = scaler.transform(X)
    scaled_X_train  = pd.DataFrame(scaled_X_train , columns=X_col)

    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', max_depth=5)
    nmse = cross_validate(xg_reg, scaled_X_train, y_train,
                      scoring=['neg_mean_squared_error'],
                      return_train_score=True,
                      cv=5, n_jobs=-1)
    mse = -1 * nmse['test_neg_mean_squared_error']
    rmse = np.sqrt(mse)
    avg_rmse = round(np.mean(rmse),4)
    print(f"평균 RMSE :{avg_rmse :.4f}")
    return avg_rmse

In [7]:
# train.csv 파일 불러오기
train_data = pd.read_csv('train.csv')

new_columns = {
    '섭씨 온도(°⁣C)': '섭씨온도',
    '절대 온도(K)': '절대온도',
    '이슬점 온도(°C)': '이슬점온도',
    '상대 습도 (%)': '상대습도',
    '대기압(mbar)': '대기압',
    '포화 증기압(mbar)': '포화증기압',
    '실제 증기압(mbar)': '실제증기압',
    '증기압 부족량(mbar)': '증기압부족량',
    '수증기 함량 (g/kg)': '수증기함량',
    '공기 밀도 (g/m**3)': '공기밀도',
    '풍향 (deg)': '풍향',
    '풍속 (m/s)': '풍속'
}

# 열 이름 변경 적용
train_data.rename(columns=new_columns, inplace=True)
# 결측치를 평균값으로 대체
train_data = train_data.fillna(train_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
encoder = OneHotEncoder(sparse=False)
time_encoded = encoder.fit_transform(train_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
train_data = pd.concat([train_data, time_encoded_df], axis=1).drop(['측정 시간대','ID','월','일'], axis=1)

train_data = train_data.drop(['상대습도', '대기압', '공기밀도', '측정 시간대_새벽', '측정 시간대_오전', '측정 시간대_오후', '측정 시간대_저녁'], axis=1)

# 경우의 수를 담을 리스트 생성
new_col = ['섭씨온도*절대온도*이슬점온도*포화증기압','섭씨온도*절대온도*이슬점온도*실제증기압','섭씨온도*절대온도*이슬점온도*증기압부족량','섭씨온도*절대온도*이슬점온도*수증기함량','실제증기압*수증기함량']        
train_data = get_new_col(train_data, new_col)


C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
# 풍속을 예측할 특성(입력 변수)과 풍속(출력 변수)을 분리합니다.
X_train = train_data.drop(['풍속'], axis=1)  # 입력 변수들
y_train = train_data['풍속']  # 출력 변수 (풍속)


In [9]:
# XGBoost 모델 정의
xgb_model = XGBRegressor(objective ='reg:squarederror', random_state=42)

# 탐색할 하이퍼파라미터 값들을 딕셔너리 형태로 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 탐색
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 값과 그 때의 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score (Negative MSE):", grid_search.best_score_)

Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.8}
Best Score (Negative MSE): -0.9751267923963207


In [10]:
# 최적의 하이퍼파라미터로 모델 재학습
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)


XGBRegressor(colsample_bytree=1.0, max_depth=7, n_estimators=300,
             objective='reg:squarederror', random_state=42, subsample=0.8)

In [12]:
# test.csv 파일 불러오기
test_data = pd.read_csv('test.csv')
# 열 이름 변경 적용
test_data.rename(columns=new_columns, inplace=True)
# 결측치를 평균값으로 대체
test_data = test_data.fillna(test_data.mean())

# '측정 시간대'를 원핫 인코딩하여 숫자 형태로 변환
time_encoded = encoder.transform(test_data[['측정 시간대']])
time_encoded_df = pd.DataFrame(time_encoded, columns=encoder.get_feature_names(['측정 시간대']))
test_data = pd.concat([test_data, time_encoded_df], axis=1).drop(['측정 시간대','ID','월','일'], axis=1)

test_data = test_data.drop(['상대습도', '대기압', '공기밀도', '측정 시간대_새벽', '측정 시간대_오전', '측정 시간대_오후', '측정 시간대_저녁'], axis=1)
test_data = get_new_col(test_data, new_col)

# 테스트 데이터로 풍속 예측을 수행합니다.
X_test = test_data  # 테스트 입력 변수들
y_pred = best_model.predict(X_test)  # 테스트 데이터로 풍속 예측

C:\Users\user\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
C:\Users\user\anaconda3\envs\tf\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
# Submit / 제출
submission = pd.read_csv('./sample_submission.csv')
submission['풍속 (m/s)'] = y_pred

# 예측 결과를 submission.csv 양식에 맞게 저장합니다.
submission.to_csv('submission.csv', index=False)

print("풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.")

풍속 예측이 완료되었습니다. 결과가 submission.csv에 저장되었습니다.


In [ ]:
# Cross Validation을 활용하여 평균 RMSE 계산
scores = cross_val_score(best_model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
rmse_scores = np.sqrt(-scores)

print("평균 RMSE:", np.mean(rmse_scores))